<a href="https://colab.research.google.com/github/adidror005/canvas/blob/main/Copy_of_TradingTheNewsWithAlpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install alpaca-py

In [ ]:
!pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 2.0 MB/s eta 0:00:00


### Since Running Asyncio in Colab/Jupyer

In [ ]:
import nest_asyncio
nest_asyncio.apply()

### Load Alpaca API Key + Secret

In [ ]:
from google.colab import userdata
ALPACA_API_KEY=userdata.get('ALPACA_API_KEY')
ALPACA_API_SECRET=userdata.get('ALPACA_API_SECRET')

### Setup OpenAI Classifier

In [ ]:
import openai
from openai import OpenAI
import json
from textwrap import dedent

# OpenAI API key
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)



# Define the financial classification prompt
classification_prompt = '''
    You are a financial market expert tasked with classifying news articles into one of three categories: "Buy", "Sell", or "Hold".
    Your decision should be based solely on the headline and summary provided below.

    Consider the following:
    1. Evaluate the tone and content of the news to gauge market sentiment.
    2. "Buy" indicates strong positive sentiment, suggesting potential for the stock to increase in value.
    3. "Sell" indicates strong negative sentiment, suggesting potential for the stock to decrease in value.
    4. "Hold" indicates neutral or unclear sentiment.

    Headline and Summary:
    - Headline: {headline}
    - Summary: {summary}

    Rules:
    - Use clear reasoning based on the content provided.
    - If sentiment cannot be determined or is ambiguous, default to "Hold".
'''

# Function to classify sentiment using structured outputs
def classify_news_with_structured_output(headline, summary):
    prompt = dedent(classification_prompt.format(headline=headline, summary=summary))

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {
                "role": "system",
                "content": prompt
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "news_classification",
                "schema": {
                    "type": "object",
                    "properties": {
                        "classification": {
                            "type": "string",
                            "enum": ["Buy", "Sell", "Hold"]
                        },
                        "reasoning": {
                            "type": "string"
                        }
                    },
                    "required": ["classification", "reasoning"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )

    message_content = response.choices[0].message.content
    data = json.loads(message_content)
    classification = data.get("classification", "Hold")
    reasoning = data.get("reasoning", "No reasoning provided.")

    return classification, reasoning

# Example Usage
headline = "Tech Giant Reports Record-Breaking Earnings for Q4"
summary = "The company's latest earnings report showed a 25% increase in revenue and 30% growth in net profit, driven by strong demand for its flagship products."



### Write an Event Handler Function For New News

In [ ]:
from alpaca.data.models.news import News
news_list=[]
async def on_new_news(news_item:News):
    # Check if only 1 symbol
    if len(news_item.symbols)==1:
      print("-----------------------------------------------------")
      print("-----------------------------------------------------")
      print("New Article")
      print("-----------------------------------------------------")
      print(news_item.headline)
      print("-----------------------------------------------------")

      classifcation,reasoning  = classify_news_with_structured_output(news_item.headline,news_item.summary)
      if classifcation=='Buy':
        print(f"BUY 1000$ of {news_item.symbols[0]}")
        print("-----------------------------------------------------")

      print(f"{classifcation}:{reasoning}")
      print("-----------------------------------------------------")
      print("-----------------------------------------------------")
      print()
      print()

    global news_list
    news_list.append(news_item)


In [ ]:
from alpaca.data.models.news import News
from IPython.display import display, HTML

news_list = []

async def on_new_news(news_item: News):
    """
    Processes new news items, classifying sentiment and printing results.

    Args:
        news_item: The News object containing news data.
    """
    classification, reasoning = classify_news_with_structured_output(
        news_item.headline, news_item.summary
    )

    # Format the output using HTML
    output = f"""
    <h3>New Article</h3>
    <hr>
    <h4>{news_item.headline}</h4>
    <p><b>{classification}</b>: {reasoning}</p>
    """

    if classification == 'Buy':
        output += f"<p><b>BUY 1000$ of {news_item.symbols[0]}</b></p>"

    output += "<hr>"

    display(HTML(output)) # Display the formatted output

    global news_list
    news_list.append(news_item)

### Initialize News Stream

In [ ]:
from alpaca.data.live import NewsDataStream
news_stream = NewsDataStream(ALPACA_API_KEY,ALPACA_API_SECRET)#,raw_data=True)

### Subscribe to News

In [ ]:
large_cap_stocks = [
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'TSLA', 'META', 'NVDA', 'BRK.B', 'UNH',
    'JNJ', 'JPM', 'V', 'PG', 'XOM', 'HD', 'CVX', 'MA', 'BAC', 'PFE',
    'ABBV', 'KO', 'PEP', 'AVGO', 'COST', 'CSCO', 'WMT', 'MCD', 'DIS', 'MRK',
    'AMD', 'INTC', 'T', 'VZ', 'WFC', 'CRM', 'ABT', 'CMCSA', 'NKE', 'ADBE',
    'LIN', 'PM', 'NEE', 'TXN', 'DHR', 'LOW', 'UNP', 'UPS', 'HON',
    'QQQ', 'SPY'
]
news_stream.subscribe_news(on_new_news,*large_cap_stocks)


In [ ]:
print("**afd**")

**afd**


In [ ]:
news_stream.run()

keyboard interrupt, bye


TimeoutError: 

In [ ]:
import asyncio
asyncio.create_task(news_stream._run_forever())

<Task pending name='Task-1' coro=<DataStream._run_forever() running at /usr/local/lib/python3.10/dist-packages/alpaca/data/live/websocket.py:315>>

In [ ]:
import pandas as pd
from google.colab import files
pd.DataFrame([n.__dict__ for n in news_list]).to_csv('rt_news_sample.csv',index=False)
files.download('rt_news_sample.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
len(news_list)

38

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
l=[classify_news_with_structured_output(n.headline,n.summary) for n in news_list if 'WFC' in n.symbols and len(n.symbols)==1][0]

ADI
ADI
ADI


KeyboardInterrupt: 

In [ ]:
len(news_list)

37

ADI


In [ ]:
reasoning

"The headline and summary indicate a strong positive sentiment due to record-breaking earnings, a significant increase in revenue and net profit, and strong demand for the company's products. This suggests that the company's stock is likely to increase in value."

In [ ]:
news_stream.run()

New news item: id=42995131 headline='Elon Musk&#39;s Tesla Leads The Pack In Top-Selling EVs In US In 2024: Ford, GM Follow' source='benzinga' url='https://www.benzinga.com/25/01/42995131/10-best-selling-evs-in-us-in-2024' summary='Over 1.3 million EVs were sold in the U.S. in 2024, marking a growth of 7.3% year-on-year, and accounting for 8.1% of total auto sales.' created_at=datetime.datetime(2025, 1, 15, 8, 3, 9, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 1, 15, 8, 3, 10, tzinfo=datetime.timezone.utc) symbols=['F', 'TSLA'] author='Anan Ashraf' content='<p>Over 1.3 million electric vehicles were sold in the U.S. in 2024, marking a growth of 7.3% year-on-year, and accounting for 8.1% of total auto sales, according to data from automotive research company <strong>Kelley Blue Book</strong>. </p>\n\n\n\n<p><strong>What Happened:</strong> EV sales in the U.S. benefited from strong incentives from automakers, excellent lease deals, and federal and state incentive prog